In [ ]:
def mount_adls(container_name, storage_account, client_id, tenant_id, secret):
    configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": f"{client_id}",
           "fs.azure.account.oauth2.client.secret": f"{secret}",
           "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    dbutils.fs.mount(
    source = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/",
    mount_point = f"/mnt/{storage_account}/{container_name}",
    extra_configs = configs)

In [ ]:
#storage_account = "retailadls218"
#client_id            = dbutils.secrets.get(scope="retail-scope", key="sp-client-id")
#tenant_id            = dbutils.secrets.get(scope="retail-scope", key="sp-tenant-id")
#client_secret        = dbutils.secrets.get(scope="retail-scope", key="sp-secret-value")

#mount_adls('rejects',storage_account,client_id,tenant_id,client_secret )

In [ ]:
#dbutils.fs.mounts()

In [ ]:
def read_df(vformat, vpath):
    df = spark.read \
                .format(vformat) \
                .option("header",True) \
                .option("inferSchema", True) \
                .option("path",vpath) \
                .load()
    return df

In [ ]:
def read_df_w_schema(vformat,schema, vpath):
    df = spark.read \
                .format(vformat) \
                .option("header",True) \
                .option("schema", schema) \
                .option("path",vpath) \
                .load()
    return df

In [ ]:
def read_df_nested_json(schema, vpath):
    df = spark.read \
                .format('json') \
                .option("schema", schema) \
                .option("multiline",True) \
                .option("path",vpath) \
                .load()
    return df

In [ ]:
#display(read_df('csv','/mnt/retailadls218/raw/orders.csv'))

order_id,order_date,customer_id,order_status
1111111,00:00.0,11599,CLOSED
2222222,00:00.0,256,PENDING_PAYMENT
1111111,00:00.0,12111,COMPLETE
4444444,00:00.0,8827,CLOSED


In [ ]:
def read_sql_table(vserver, vport, vDBName, vDBUser, Vpassword, vtable):
    connectionUrl='jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(vserver,vport,vDBName,vDBUser)

    connectionProperties={
                        'password':Vpassword,
                        'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
                        }
    
    sql_DF = spark.read \
                    .jdbc( url=connectionUrl,
                           table=vtable,
                           properties=connectionProperties
                         )
    return sql_DF



In [ ]:
def write_w_part(df, vformat, vpath, vpartcol, vmode, vtable):
    df.write \
        .option("header",True) \
        .format(vformat) \
        .option("path",vpath) \
        .partitionBy(vpartcol) \
        .mode(vmode) \
        .saveAsTable(vtable )
    return df

In [ ]:
def write_df(df, vformat, vpath, vmode):
    df.write \
        .option("header",True) \
        .format(vformat) \
        .option("path",vpath) \
        .mode(vmode) \
        .save()
    return df